# Geocoding, Self-contained example

This script takes in google analytics city name and region columns, cleans them, and feeds it into several API's to geocode (API-keys need to be provided in the dummy_key.py file). 

The end-goals here is to create a clean, geocoded file of all the cities that appear in the set of google analytics data fed into it. This could then be used to merge with the original or new data to add socio-economical data that is geographically granular.

We also show how one can merge with Eurostat NUTS codes.

## load packages

Needed packages: 

- geopy
- pandas
- matplotlib
- numpy
- basemap

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemapimport
import pandas as pd

# import the geocoding services you'd like to try
from geopy.geocoders import ArcGIS, Bing, Nominatim, OpenCage, GoogleV3, OpenMapQuest
from time import sleep
from geopy.extra.rate_limiter import RateLimiter
import csv
import dummy_keys #change this with the file with your own dummy keys inserted

## load data

We first load in an example file, where we extracted the city name and region name from a real google analytics dataset.

In [ ]:
df = pd.read_csv('example_google_analytics_city_region_file.csv', index_col = 0)

In [ ]:
len(df)

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.head()

We then take a list of all cities/regions that can appear in the google analytics dataset. This file, and newer versions of it can be found here: *https://developers.google.com/analytics/devguides/collection/protocol/v1/geoid*.

In [ ]:
google_cities = pd.read_csv('geotargets_google.csv', index_col=0)
google_cities = google_cities[['Name','Canonical Name','Country Code','Target Type','Status']]

*Note: here it might be useful to subselect the google_cities with target type 'City', however, we also found cantons and other types appearing in the raw df data so I continue with all of them. The main downside of this is that certain Names might appear double (for instance one for the Paris as a city and one for region). However, since the canonical name for these different types will still be paris, the API **should** still find the right location for these names.*

In [ ]:
google_cities.head()

We use this file to reconstruct the canonical name, which we will feed into the geocoding, from just the city and region names that we got from our google analytics file.

### Construct country and region names in from canonical name in google list

In [ ]:
google_cities['Country Name'] = google_cities['Canonical Name'].str.split(',').str[-1]
google_cities['Region Name'] = google_cities['Canonical Name'].str.split(',').str[-2]

### Now we can find the overlapping cities between those in df and those in google_cities

In [ ]:
merge_cities = google_cities.merge(df, on = ['Region Name','Name'], how = 'right')

In [ ]:
len(merge_cities)

In [ ]:
merge_cities.tail()

We see that this first pass still left many regions in our data unmatched with their google counterpart. We now try to fix these issues by cleaning the region names in our data.

# Fix regions

We initialize a new 'Clean Region Name' column which we will utilize to try and clean the original region names that were hindering a smooth merge.

In [ ]:
merge_cities['Clean Region Name'] = np.nan

### Fix regions in England

We start with Englang and more specifically with what seem to be TV Regions in the google data, which are fictious regions that for some reason show up in the data.

In [ ]:
google_cities[google_cities['Target Type']=='TV Region'].head()

In [ ]:
regions_uk=['South East',
'North West',
'London',
'East of England',
'West Midlands',
'Midlands',
'South West',
'Yorkshire',
'East Midlands',
'North East']
merge_cities.loc[merge_cities['Region Name'].isin(regions_uk), 'Country Name'] = 'United Kingdom'
merge_cities.loc[merge_cities['Region Name'].isin(regions_uk), 'Country Code'] = 'GB'
merge_cities.loc[merge_cities['Region Name'].isin(regions_uk), 'Clean Region Name'] = merge_cities['Region Name'] + ',England'

#TV regions
tv_regions = google_cities[google_cities['Target Type']=='TV Region']
tv_loc = merge_cities['Region Name'].isin(list(tv_regions.Name))
tv_loc_dict = tv_regions[['Name','Region Name']].set_index('Name').to_dict()['Region Name']

merge_cities.loc[tv_loc, 'Country Name'] = 'United Kingdom'
merge_cities.loc[tv_loc, 'Country Code'] = 'GB'
merge_cities.loc[tv_loc, 'Target Type'] = 'TV Region'
for i in tv_loc_dict.keys():
    merge_cities.loc[((tv_loc)&(merge_cities['Region Name'] == i)),'Clean Region Name'] = tv_loc_dict[i]

merge_cities.loc[merge_cities['Region Name'].str.contains(" \(exc. Channel Islands\)"),"Clean Region Name"]= merge_cities["Region Name"].str.replace(" \(exc. Channel Islands\)", "", case = True)
merge_cities.loc[merge_cities['Region Name'].str.contains("HTV"),"Clean Region Name"]= merge_cities["Region Name"].str.replace("HTV ", "", case = True)

merge_cities.loc[((merge_cities['Country Code'] == 'GB') & (merge_cities['Canonical Name'].isnull())), 'Canonical Name'] = merge_cities.Name + ',' + merge_cities['Clean Region Name'] + ',' + merge_cities['Country Name']

### Fix american state appendices

Next we move on to the american states. It seems that in our cities, sometime the abbreviation for the state is added, which hinders the merge. Below we show an example for Chicago, where the abbreviation IL for Illinois is added.

In [ ]:
merge_cities[merge_cities['Region Name'].str.contains('IL')].head()

In [ ]:
US_state_full = {'AL': 'Alabama',
 'AK': 'Alaska',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'DC': 'District of Columbia',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'MP': 'Northern Mariana Islands',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PW': 'Palau',
 'PA': 'Pennsylvania',
 'PR': 'Puerto Rico',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VI': 'Virgin Islands',
 'VA': 'Virginia',
 'WA': 'Washington',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming'}

american_States_space_sep = ((merge_cities['Canonical Name'].isnull())&(merge_cities['Region Name'].str.split(' ').str[-1].str.isupper())&(merge_cities['Region Name'].str.split(' ').str[-1].str.isalpha())& (merge_cities['Region Name'].str[-2:].str.isalpha()))

merge_cities.loc[american_States_space_sep,'Country Name'] = 'United States'
merge_cities.loc[american_States_space_sep,'Country Code'] = 'US'
merge_cities.loc[american_States_space_sep,'Clean Region Name'] = merge_cities['Region Name'].str[-2:]

american_States_comma_sep = ((merge_cities['Canonical Name'].isnull())&(merge_cities['Region Name'].str.split(',').str[-1].str.isupper())&(merge_cities['Region Name'].str.split(',').str[-1].str.isalpha())& (merge_cities['Region Name'].str[-2:].str.isalpha()))

merge_cities.loc[american_States_comma_sep,'Country Name'] = 'United States'
merge_cities.loc[american_States_comma_sep,'Country Code'] = 'US'
merge_cities.loc[american_States_comma_sep,'Clean Region Name'] = merge_cities['Region Name'].str[-2:]

for i in US_state_full.keys():
    merge_cities.loc[merge_cities['Country Code']=='US','Clean Region Name'] = merge_cities['Clean Region Name'].str.replace(i,US_state_full[i])

merge_cities.loc[((merge_cities['Country Code']=='US') & (merge_cities['Canonical Name'].isnull())), 'Canonical Name'] = merge_cities.Name + ',' + merge_cities['Clean Region Name'] + ',' + merge_cities['Country Name']

dc = merge_cities['Region Name'] == 'Washington DC (Hagerstown MD)'
merge_cities.loc[dc,'Country Name'] = 'United States'
merge_cities.loc[dc,'Country Code'] = 'US'
merge_cities.loc[dc,'Clean Region Name'] = 'Washington DC'
merge_cities.loc[dc,'Canonical Name'] = merge_cities.Name + ',' + merge_cities['Clean Region Name'] + ',' + merge_cities['Country Name']

In [ ]:
merge_cities[merge_cities['Region Name'].str.contains('IL')].head()

## Cantons in Switserland

The next peculiarity we tackle are the Cantons in Switserland, where for some reason we have a quite useless prefix 'Canton of'.

In [ ]:
merge_cities[(merge_cities['Canonical Name'].isnull())&(merge_cities['Region Name'].str.contains('Canton of'))].head()

In [ ]:
cantons = (merge_cities['Canonical Name'].isnull())&(merge_cities['Region Name'].str.contains('Canton of'))
merge_cities.loc[cantons,'Country Name'] = 'Switzerland'
merge_cities.loc[cantons,'Country Code'] = 'CH'
merge_cities.loc[cantons,'Canonical Name'] = merge_cities.Name + ',' + merge_cities['Region Name'] + ',' + merge_cities['Country Name']

In [ ]:
merge_cities[merge_cities['Region Name'].str.contains('Neuchatel')].head()

## Fix JP_ prefix for Japan

Next, I found that for Japan, there is a JP_ prefix. So let's fix this.

In [ ]:
merge_cities[merge_cities['Region Name'].str.contains('JP_')].head()

In [ ]:
japan = (merge_cities['Canonical Name'].isnull())&(merge_cities['Region Name'].str.contains('JP_'))
merge_cities.loc[japan,'Country Name'] = 'Japan'
merge_cities.loc[japan,'Country Code'] = 'JP'
merge_cities.loc[japan&~(merge_cities['Region Name'].str.split('_').str[-1] == 'OTHER'),'Clean Region Name'] = merge_cities['Region Name'].str.split('_').str[-1]
merge_cities.loc[japan,'Canonical Name'] = merge_cities.Name + ',' + merge_cities['Clean Region Name'] + ',' + merge_cities['Country Name']
merge_cities.loc[japan&(merge_cities['Region Name'].str.split('_').str[-1] == 'OTHER'),'Canonical Name'] = merge_cities.Name + ',' + merge_cities['Country Name']

In [ ]:
merge_cities[merge_cities['Region Name'].str.contains('JP_')].head()

### For those cities not found merge name and region name

For the remaining city, we just have to kind of hope that the canonical name we compile from available information is precise enough so that the API's can give us their best guess of where these locations are.

In [ ]:
overlap_places = merge_cities[~merge_cities['Canonical Name'].isnull()]
non_overlap_places = merge_cities[merge_cities['Canonical Name'].isnull()]

for i in list(non_overlap_places.Name):
    if i in list(overlap_places.Name):
        merge_cities.loc[((merge_cities['Name']==i)&(merge_cities['Canonical Name'].isnull())),'Country Name'] = overlap_places.loc[overlap_places.Name == i,'Country Name'].values[0]
        merge_cities.loc[((merge_cities['Name']==i)&(merge_cities['Canonical Name'].isnull())),'Country Code'] = overlap_places.loc[overlap_places.Name == i,'Country Code'].values[0]
        merge_cities.loc[((merge_cities['Name']==i)&(merge_cities['Canonical Name'].isnull())&(merge_cities['Clean Region Name'].isnull())),'Canonical Name'] = merge_cities.Name + ',' + merge_cities['Region Name'] + ',' + merge_cities['Country Name']
        merge_cities.loc[((merge_cities['Name']==i)&(merge_cities['Canonical Name'].isnull())&~(merge_cities['Clean Region Name'].isnull())),'Canonical Name'] = merge_cities.Name + ',' + merge_cities['Clean Region Name'] + ',' + merge_cities['Country Name']

In [ ]:
overlap_places = merge_cities[~merge_cities['Canonical Name'].isnull()]
non_overlap_places = merge_cities[merge_cities['Canonical Name'].isnull()]

non_overlap_places['Canonical Name'] = non_overlap_places.Name.astype(str) + ',' + non_overlap_places['Region Name'].astype(str)

In [ ]:
overlap_places.to_csv('overlap_regions.csv', encoding='utf-8', index=False)
non_overlap_places.to_csv('non_overlap_regions.csv', encoding='utf-8', index=False)

Here we write out two files, one for those we are pretty sure our description is accurate, and for those where we are less sure. I do this because for instance the Google API can handle such uncertain name a lot better than other API's. But since the free version has a limit on the amount of requests, we would request these first before running out of free requests. The more certain ones can then be requested from several API's.

## Geocoding

### Initializing API's

Here it is *Key* to have a keys.py file in the folder that contains your personal API-keys.

In [ ]:
geocoded = "geocoded/"
gc_directory = os.path.dirname(geocoded)
if not os.path.exists(gc_directory):
    os.makedirs(gc_directory)

rv_geocoded = "reverse_geocoded/"
rv_directory = os.path.dirname(rv_geocoded)
if not os.path.exists(rv_directory):
    os.makedirs(rv_directory)

in_file = str('non_overlap_regions.csv')
#add gc_ to geocoded file name
out_file = str('gc_' + in_file)
timeout = 100

print('creating geocoding objects.')

arcgis = ArcGIS(timeout=timeout)
bing = Bing(api_key=keys.bing_api,timeout=timeout)
nominatim = Nominatim(user_agent='application_n', timeout=timeout)
opencage = OpenCage(api_key=keys.oc_api,timeout=timeout)
googlev3 = GoogleV3(api_key=keys.g3_api, domain='maps.googleapis.com', timeout=timeout)
openmapquest = OpenMapQuest(api_key=keys.omq_api, timeout=timeout)

# choose and order your preference for geocoders here
geocoders = [googlev3,nominatim, bing, openmapquest, opencage, arcgis]

### Geocoding function

#### (Expects a 'Canonical Name' column as google analytics uses)

In [ ]:
def gc(row):
    add_concat = row['Canonical Name']
    for gcoder in geocoders:
        location = gcoder.geocode(add_concat)
        if location != None:
            print(f'geocoded record {add_concat}')
            located = pd.Series({
                'lat': location.latitude,
                'lng': location.longitude
            })
        else:
            print(f'failed to geolocate record {add_concat}')
            located = pd.Series({
                'lat': 'null',
                'lng': 'null'
            })
        return located

### Reverse Geocoding function nominatim

#### Expects a lat and lng column

In [ ]:
def reverse_gc(row):
    sleep(0.8) # to prevent overloading the API and getting too many request errors
    add_concat = str(row['lat']) + ',' + str(row['lng'])
    gcoder = True
    #for gcoder in geocoders:
    reverse = RateLimiter(nominatim.reverse, min_delay_seconds=0.7)
    location = reverse(add_concat, language='en', exactly_one = True)
    if location != None:
        print(f'reverse geocoded record {add_concat}')
        city = 'null'
        if 'city' in location.raw['address'].keys():
            city = location.raw['address']['city']
        elif 'town' in location.raw['address'].keys():
            city = location.raw['address']['town']
        elif 'village' in location.raw['address'].keys():
            city = location.raw['address']['village']
        elif 'suburb' in location.raw['address'].keys():
            city = location.raw['address']['suburb']
        elif 'hamlet' in location.raw['address'].keys():
            city = location.raw['address']['hamlet']
                    
        county = 'null'
        if 'county' in location.raw['address'].keys():
            county = location.raw['address']['county']

        state = 'null'
        if 'state' in location.raw['address'].keys():
            state = location.raw['address']['state']

        region = 'null'
        if 'region' in location.raw['address'].keys():
            region = location.raw['address']['region']

        postcode = 'null'
        if 'postcode' in location.raw['address'].keys():
            postcode = location.raw['address']['postcode']

        country = 'null'
        if 'country' in location.raw['address'].keys():
            country = location.raw['address']['country']
            
        country_code = 'null'
        if 'country_code' in location.raw['address'].keys():
            country_code = location.raw['address']['country_code']
     

        located = pd.Series({
            'city_reverse': city,
            'county_reverse': county,
            'state_reverse': state,
            'region_reverse': region,
            'postcode_reverse': postcode,
            'country_reverse': country,
            'country_code_reverse': country_code
        })

    else:
        print(f'failed to reverse geolocate record {add_concat}')
        located = pd.Series({
            'city_reverse': 'null',
            'county_reverse': 'null',
            'state_reverse': 'null',
            'region_reverse': 'null',
            'postcode_reverse': 'null',
            'country_reverse': 'null',
            'country_code_reverse': 'null'
        })
    return located

### Geocoding csv file in chunks to avoid data loss if connection failure

In [ ]:
print('opening input.')
reader = pd.read_csv(in_file, header=0)
nr_chunks = 0
#if your file is too large we split it up into files of around 500 entries to avoid too much loss of progress when errors are thrown
if len(reader) > 800:
    nr_chunks = int(np.floor(len(reader)/500))
    chunks = np.array_split(reader, nr_chunks)
    print(f'Splitted data into {nr_chunks} chunks.')
    i = 1
    for chunk in chunks:
        print(f'starting geocoding on chunk {i}')
        try:
            #this again uses the same order as your original list of API's, here the google-API is tried first
            chunk = chunk.merge(chunk.apply(lambda add: gc(add), axis=1), left_index=True, right_index=True)
        except Exception as e:
            print(f'Caught exception, stopping geocoding. See geocode_error.txt for more info.')
            file = open('geocode_error.txt','w') 
            file.write(f"Error occured at chunk {i}") 
            file.write(f"The error-code was:'{str(e)}'") 
            file.close()
            
        out_file_chunk = str(f'chunk_{i}_of_{nr_chunks}_' + out_file)
        print(f'writing chunk {i} to {out_file_chunk}.')
        chunk.to_csv(geocoded+out_file_chunk, encoding='utf-8', index=False)
        i+=1
    print('done.')

else:
    print('geocoding addresses.')
    reader = reader.merge(reader.apply(lambda add: gc(add), axis=1), left_index=True, right_index=True)
    print(f'writing to {out_file}.')
    reader.to_csv(geocoded+out_file, encoding='utf-8', index=False)
    print('done.')

# From the geocoded file we can now revert back to get the postcodes, which is more useful since government data is often on this level.

## Inverting from coordinates to places and POSTCODES for all the chunks

In [ ]:
from time import sleep
from geopy.extra.rate_limiter import RateLimiter

if nr_chunks>0:
    for i in range(1,nr_chunks+1):
        in_file_chunk = str(f'chunk_{i}_of_{nr_chunks}_' + out_file)
        print(f'opening chunk {i}.')
        reader = pd.read_csv(in_file_chunk, header=0)
        print('reverse geocoding addresses.')
        reader = reader.merge(reader.apply(lambda add: reverse_gc(add), axis=1), left_index=True, right_index=True)
        out_file_chunk = str('rv_' + in_file_chunk)
        print(f'writing to {out_file_chunk}.')
        reader.to_csv(rv_geocoded+out_file_chunk, encoding='utf-8', index=False)
        print('done.')
else:
    print(f'opening file.')
    reader = pd.read_csv(out_file, header=0)
    print('reverse geocoding addresses.')
    reader = reader.merge(reader.apply(lambda add: reverse_gc(add), axis=1), left_index=True, right_index=True)
    out_file_rv = str('rv_' + out_file)
    print(f'writing to {out_file_rv}.')
    reader.to_csv(rv_geocoded+out_file_rv, encoding='utf-8', index=False)
    print('done.')

## Googlev3 reverse encoding API (Alternative for Nomatim above)

In [ ]:
def get_google_component(location, component_type, long_name=True):
    for component in location.raw['address_components']:
        if component_type in component['types']:
            if long_name:
                return component['long_name']
            else:
                return component['short_name']


def google_reverse_gc(row):
    add_concat = str(row['lat']) + ',' + str(row['lng'])
    gcoder = True
    reverse = RateLimiter(googlev3.reverse, min_delay_seconds=0.7)
    location = reverse(add_concat, language='en', exactly_one = True)
    if location != None:
        print(f'reverse geocoded record {add_concat}')
        city = get_google_component(location, 'postal_town')
        county = get_google_component(location, 'administrative_area_level_2')
        region = get_google_component(location, 'administrative_area_level_1')
        post_code = get_google_component(location, 'postal_code')
        country = get_google_component(location, 'country')
        country_code = get_google_component(location, 'country', long_name=False)

        located = pd.Series({
            'city_reverse': city,
            'county_reverse': county,
            'region_reverse': region,
            'postcode_reverse': post_code,
            'country_reverse': country,
            'country_code_reverse': country_code
        })

    else:
        print(f'failed to reverse geolocate record {add_concat}')
        located = pd.Series({
            'city_reverse': 'null',
            'county_reverse': 'null',
            'region_reverse': 'null',
            'postcode_reverse': 'null',
            'country_reverse': 'null',
            'country_code_reverse': 'null'
        })
    return located

In [ ]:
from time import sleep
from geopy.extra.rate_limiter import RateLimiter

if nr_chunks>0:
    for i in range(1,nr_chunks+1):
        in_file_chunk = str(f'chunk_{i}_of_{nr_chunks}_' + out_file)
        print(f'opening chunk {i}.')
        reader = pd.read_csv(in_file_chunk, header=0)
        print('reverse geocoding addresses.')
        reader = reader.merge(reader.apply(lambda add: google_reverse_gc(add), axis=1), left_index=True, right_index=True)
        out_file_chunk = str('google_rv_' + in_file_chunk)
        print(f'writing to {out_file_chunk}.')
        reader.to_csv(rv_geocoded+out_file_chunk, encoding='utf-8', index=False)
        print('done.')
else:
    print(f'opening file.')
    reader = pd.read_csv(out_file, header=0)
    print('reverse geocoding addresses.')
    reader = reader.merge(reader.apply(lambda add: google_reverse_gc(add), axis=1), left_index=True, right_index=True)
    out_file_rv = str('google_rv_' + out_file)
    print(f'writing to {out_file_rv}.')
    reader.to_csv(rv_geocoded+out_file_rv, encoding='utf-8', index=False)
    print('done.')

# Connecting postal codes to NUTS codes

## Merge all reverse_geocoded_files

In [ ]:
import pandas as pd
import glob

path = 'reverse_geocoded/' # use your path
all_files = glob.glob(path + "/*.csv")

In [ ]:
li = []
ii=0
if nr_chunks>0:
    for filename in all_files:
        print(filename)
        df = pd.read_csv(filename,header = 0, index_col=0)
        print(len(df))
        ii+=len(df)
        li.append(df)

    frame = pd.concat(li, axis=0)
    print(ii)
    frame.to_csv(path+'all_reverse_geocoded.csv',encoding='utf-8')
    all_gc_path = path+'all_reverse_geocoded.csv'
else:
    for filename in all_files:
        if 'google' in filename:
            all_gc_path = path+'google_rv_gc_non_overlap_regions.csv'
        else:
            all_gc_path = path+'rv_gc_non_overlap_regions.csv'

### Retry unknown postcodes with google api

In [ ]:
df = pd.read_csv(all_gc_path,header=0, index_col=None)
#df = df.reset_index().drop(columns = ['index'])
df.loc[df['Canonical Name'] == 'Oxted,England,United Kingdom', 'lng'] = 0

retry = df[df.postcode_reverse.isnull()]
df_nn = df[~df.postcode_reverse.isnull()]

reader = retry[['Canonical Name',
 'Clean Region Name',
 'Country Code',
 'Country Name',
 'Name',
 'Region Name',
 'Status',
 'Target Type',
 'lat',
 'lng',
 'state_reverse']]

print('reverse geocoding addresses.')
reader = reader.merge(reader.apply(lambda add: google_reverse_gc(add), axis=1), left_index=True, right_index=True)
df = pd.concat([df_nn,reader[~reader.postcode_reverse.isnull()]], axis=0)
df.to_csv(path+'all_reverse_geocoded_plus_retry.csv',encoding='utf-8')

At this point you have a datafile that contains all the city/region combination you fed into the processing together with all the geo-features you need to merge it with statistical region codes/data.

# Let's start merging with the NUTS code

In [ ]:
path = 'reverse_geocoded/'
df = pd.read_csv(path+'all_reverse_geocoded_plus_retry.csv',encoding='utf-8',header=0, index_col=0)

### Fix some peculiarities with different country codes between google and nuts

In [ ]:
df.country_code_reverse = df.country_code_reverse.str.lower()
df.loc[df.country_code_reverse=='gb','country_code_reverse'] = 'uk'
df.loc[df.country_code_reverse=='gr','country_code_reverse'] = 'el'
df.loc[(df.postcode_reverse.str.contains('\.')),'postcode_reverse'] = df.postcode_reverse.str[:-2]

### Read in the postal codes to nuts codes file

In [ ]:
nuts = pd.read_csv('all_nuts_to_postal_country_code.csv',encoding='utf-8',header=0, index_col=0)
nuts = nuts.dropna()

### perform the first merge try

In [ ]:
first_step = df.merge(nuts.dropna(),how='left', on=['postcode_reverse','country_code_reverse'])
first_step = first_step[first_step.country_code_reverse.isin(list(nuts.country_code_reverse.unique()))]

### checking if we can find nuts code for postcode with same prefix, which is accurate enough for NUTS3

In [ ]:
postcodes = nuts.postcode_reverse
to_do = list(first_step[first_step.NUTS3.isnull()].postcode_reverse.unique())
for i in to_do:
    if ' ' in i.strip():
        i_start = i.split(' ')[0]
        if sum(nuts.postcode_reverse.str.startswith(i_start)) > 0:
            nuts_code = nuts[postcodes.str.startswith(i_start)].NUTS3.values[0]
            first_step.loc[first_step.postcode_reverse == i, 'NUTS3'] = nuts_code

In [ ]:
nuts_path = "postcode_to_nuts/"
nuts_directory = os.path.dirname(nuts_path)
if not os.path.exists(nuts_directory):
    os.makedirs(nuts_directory)
first_step.to_csv(nuts_path+'city_to_nuts_iteration1.csv',encoding='utf-8')

### fixing german starting 0's in postcode, which don't exist in eurostats tables

In [ ]:
postcodes = nuts.postcode_reverse
to_do_de = list(first_step[(first_step.NUTS3.isnull())&(first_step.country_code_reverse == 'de')].postcode_reverse.unique())
for i in to_do_de:
    if i.strip().startswith('0'):
        i_end = i[1:]
        matches = nuts[(nuts.postcode_reverse == i_end)&(nuts.country_code_reverse == 'de')]
        if len(matches) > 0:
            nuts_code = matches.NUTS3.values[0]
            first_step.loc[first_step.postcode_reverse == i, 'NUTS3'] = nuts_code

### doing the same for spanish cities

In [ ]:
postcodes = nuts.postcode_reverse
to_do_es = list(first_step[(first_step.NUTS3.isnull())&(first_step.country_code_reverse == 'es')].postcode_reverse.unique())
for i in to_do_es:
    if i.strip().startswith('0'):
        i_end = i[1:]
        matches = nuts[(nuts.postcode_reverse == i_end)&(nuts.country_code_reverse == 'es')]
        if len(matches) > 0:
            nuts_code = matches.NUTS3.values[0]
            first_step.loc[first_step.postcode_reverse == i, 'NUTS3'] = nuts_code

## Le même pour la republique

In [ ]:
postcodes = nuts.postcode_reverse
to_do_fr = list(first_step[(first_step.NUTS3.isnull())&(first_step.country_code_reverse == 'fr')].postcode_reverse.unique())
for i in to_do_fr:
    if i.strip().startswith('0'):
        i_end = i[1:]
        matches = nuts[(nuts.postcode_reverse.str.startswith(i_end))&(nuts.country_code_reverse == 'fr')]
        if len(matches) > 0:
            nuts_code = matches.NUTS3.values[0]
            first_step.loc[first_step.postcode_reverse == i, 'NUTS3'] = nuts_code

### fixing the dutch codes which have street identifiers e.g. 3295BW (the BW is a nuissance)

In [ ]:
postcodes = nuts.postcode_reverse
to_do_nl = list(first_step[(first_step.NUTS3.isnull())&(first_step.country_code_reverse == 'nl')].postcode_reverse.unique())
for i in to_do_nl:
    if i[-2:].isalpha():
        i_cutted = i[:-2]
        matches = nuts[(nuts.postcode_reverse == i_cutted)&(nuts.country_code_reverse == 'nl')]
        if len(matches) > 0:
            nuts_code = matches.NUTS3.values[0]
            first_step.loc[first_step.postcode_reverse == i, 'NUTS3'] = nuts_code

In [ ]:
first_step.to_csv(nuts_path+'city_to_nuts_up_till_nl.csv',encoding='utf-8')

### first pass at fixing the english

In [ ]:
postcodes = nuts.postcode_reverse
to_do_uk = list(first_step[(first_step.NUTS3.isnull())&(first_step.country_code_reverse == 'uk')].postcode_reverse.unique())
for i in to_do_uk:
    i_start = i.split(' ')[0]
    matches = nuts[(nuts.postcode_reverse.str.startswith(f'{i_start} '))&(nuts.country_code_reverse == 'uk')]
    if len(matches) > 0:
        nuts_code = matches.NUTS3.values[0]
        first_step.loc[first_step.postcode_reverse == i, 'NUTS3'] = nuts_code

### second pass at fixing the english by seeing if there is a neighbouring number that matches

In [ ]:
postcodes = nuts.postcode_reverse
to_do_uk = list(first_step[(first_step.NUTS3.isnull())&(first_step.country_code_reverse == 'uk')].postcode_reverse.unique())
for i in to_do_uk:
    i_start = i.split(' ')[0]
    i_start_1 = i_start[:-1]
    if i_start[-1].isdigit():
        lastdigit = str(int(i_start[-1]) + 1)
        i_start = i_start_1 + lastdigit
        matches = nuts[(nuts.postcode_reverse.str.startswith(f'{i_start} '))&(nuts.country_code_reverse == 'uk')]
        if len(matches) > 0:
            nuts_code = matches.NUTS3.values[0]
            first_step.loc[first_step.postcode_reverse == i, 'NUTS3'] = nuts_code
        else:
            i_start = i.split(' ')[0]
            i_start_1 = i_start[:-1]
            lastdigit = str(int(i_start[-1]) -1)
            i_start = i_start_1 + lastdigit
            matches = nuts[(nuts.postcode_reverse.str.startswith(f'{i_start} '))&(nuts.country_code_reverse == 'uk')]
            if len(matches) > 0:
                nuts_code = matches.NUTS3.values[0]
                first_step.loc[first_step.postcode_reverse == i, 'NUTS3'] = nuts_code

In [ ]:
first_step.to_csv(nuts_path+'city_to_nuts_up_till_uk.csv',encoding='utf-8')

### cutting number of the french postcodes

In [ ]:
postcodes = nuts.postcode_reverse
to_do_fr = list(first_step[(first_step.NUTS3.isnull())&(first_step.country_code_reverse == 'fr')].postcode_reverse.unique())
for i in to_do_fr:
    if i[-1:].isdigit():
        i_cutted = i[:-1]
        matches = nuts[(nuts.postcode_reverse.str.startswith(i_cutted))&(nuts.country_code_reverse == 'fr')]
        if len(matches) > 0:
            nuts_code = matches.NUTS3.values[0]
            first_step.loc[first_step.postcode_reverse == i, 'NUTS3'] = nuts_code

In [ ]:
first_step.drop_duplicates(subset=['Canonical Name','NUTS3'], keep='first', inplace=True)
first_step.to_csv('city_to_nuts_final_clean.csv',encoding='utf-8')

A bit of cleaning up files

In [ ]:
import shutil
shutil.rmtree(nuts_path[:-1])

**This file can now be used to add all the geo-features collected here to your data.**

# Plotting the cities that visited the website

In [ ]:
first_step = pd.read_csv('city_to_nuts_final_clean.csv', index_col=0)

In [ ]:
lats = list(first_step.lat)
lons = list(first_step.lng)

fig = plt.gcf()
fig.set_size_inches(8, 6.5)

m = Basemap(projection='merc', \
            llcrnrlat=-80, urcrnrlat=80, \
            llcrnrlon=-180, urcrnrlon=180, \
            lat_ts=20, \
            resolution='c')

m.bluemarble(scale=0.2)   # full scale will be overkill
m.drawcoastlines(color='white', linewidth=0.2)  # add coastlines

x, y = m(lons, lats)  # transform coordinates
plt.scatter(x, y, 10, marker='o', color='Red') 

plt.show()